# Importar librerias necesarias

In [12]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import json
import pyodbc
import locale


# Inspeccionar contenido de un archivo muestra.txt

In [13]:
def inspeccionar_tcx(ruta_archivo):
    ns = {'tcx': 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2'}
    tree = ET.parse(ruta_archivo)
    root = tree.getroot()

    # Buscar nodos clave
    tiene_pasos = root.find('.//tcx:Steps', ns) is not None
    tiene_tiempo_actividad = root.find('.//tcx:TotalTimeSeconds', ns) is not None
    tiene_velocidad = root.find('.//tcx:Speed', ns) is not None
    tiene_distancia = root.find('.//tcx:DistanceMeters', ns) is not None
    tiene_fc = root.find('.//tcx:HeartRateBpm', ns) is not None
    tiene_altitud = root.find('.//tcx:AltitudeMeters', ns) is not None
    tiene_calorias = root.find('.//tcx:Calories', ns) is not None

    print(f"Archivo: {ruta_archivo}")
    print(f"Pasos: {'Sí' if tiene_pasos else 'No'}")
    print(f"Tiempo de actividad: {'Sí' if tiene_tiempo_actividad else 'No'}")
    print(f"Velocidad: {'Sí' if tiene_velocidad else 'No'}")
    print(f"Distancia: {'Sí' if tiene_distancia else 'No'}")
    print(f"Frecuencia cardíaca: {'Sí' if tiene_fc else 'No'}")
    print(f"Altitud: {'Sí' if tiene_altitud else 'No'}")
    print(f"Calorías: {'Sí' if tiene_calorias else 'No'}")

# Extraer los valores muestra de un archivo, para filtrar o clasificar los archivos antes de analizarlos. (Bicicleta)


In [14]:

inspeccionar_tcx(r'C:\Users\ilove\OneDrive\Escritorio\Cursos Data\Basicos_Python\archivos TXC proyecto\2016-09-11T07_34_14-05_00_PT2H18M29.806S_Ciclismo_.tcx')

Archivo: C:\Users\ilove\OneDrive\Escritorio\Cursos Data\Basicos_Python\archivos TXC proyecto\2016-09-11T07_34_14-05_00_PT2H18M29.806S_Ciclismo_.tcx
Pasos: No
Tiempo de actividad: Sí
Velocidad: No
Distancia: Sí
Frecuencia cardíaca: No
Altitud: No
Calorías: Sí


In [15]:
# Segundo archivo muestra (Correr)
inspeccionar_tcx(r'C:\Users\ilove\OneDrive\Escritorio\Cursos Data\Basicos_Python\archivos TXC proyecto\2016-01-08T16_16_37-06_00_PT25M22S_Correr.tcx')

Archivo: C:\Users\ilove\OneDrive\Escritorio\Cursos Data\Basicos_Python\archivos TXC proyecto\2016-01-08T16_16_37-06_00_PT25M22S_Correr.tcx
Pasos: No
Tiempo de actividad: Sí
Velocidad: No
Distancia: Sí
Frecuencia cardíaca: No
Altitud: Sí
Calorías: Sí


# Función extraer datos clave de cada archivo, devuelve un resumen con:
- 	Fecha y hora de inicio
- 	Fecha y hora de fin
- 	Duración total
- 	Distancia total
- 	Velocidad promedio
- 	Calorias

In [16]:

def resumen_tcx(ruta_archivo):
    ns = {'tcx': 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2'}
    try:
        tree = ET.parse(ruta_archivo)
        root = tree.getroot()

        # Tipo de deporte
        actividad = root.find('.//tcx:Activity', ns)
        tipo_deporte = actividad.attrib.get('Sport') if actividad is not None else 'Desconocido'

        # Calorías (sumar todas las laps si hay más de una)
        laps = root.findall('.//tcx:Lap', ns)
        calorias = sum([
        round(float(lap.find('tcx:Calories', ns).text))
        for lap in laps
        if lap.find('tcx:Calories', ns) is not None
        ])

        # Trackpoints
        trackpoints = root.findall('.//tcx:Trackpoint', ns)
        tiempos = []
        distancias = []

        for tp in trackpoints:
            tiempo = tp.find('tcx:Time', ns)
            distancia = tp.find('tcx:DistanceMeters', ns)

            if tiempo is not None:
                tiempos.append(pd.to_datetime(tiempo.text).tz_localize(None))
            if distancia is not None:
                distancias.append(float(distancia.text))

        if len(tiempos) < 2 or len(distancias) < 2:
            return None

        inicio = min(tiempos)
        fin = max(tiempos)
        duracion_seg = (fin - inicio).total_seconds()
        distancia_total_m = max(distancias) - min(distancias)
        velocidad_prom_kmh = (distancia_total_m / duracion_seg) * 3.6
        mes = inicio.strftime('%Y-%m')

        return {
            'Archivo': os.path.basename(ruta_archivo),
            'Tipo de deporte': tipo_deporte,
            'Inicio': inicio.strftime('%d/%m/%Y %H:%M:%S'),
            'Fin': fin.strftime('%d/%m/%Y %H:%M:%S'),
            'Duración (min)': round(duracion_seg / 60, 2),
            'Distancia total (km)': round(distancia_total_m / 1000, 2),
            'Velocidad promedio (km/h)': round(velocidad_prom_kmh, 2),
            'Calorías': calorias
        }
    except Exception as e:
        print(f"Error en {ruta_archivo}: {e}")
        return None

# Procesar archivos

In [17]:
carpeta = r'C:\Users\ilove\OneDrive\Escritorio\Archivos para proyectos\complementos'  # Ajusta esta ruta
archivos = [f for f in os.listdir(carpeta) if f.endswith('.tcx')]

resumenes = []
for archivo in archivos:
    ruta = os.path.join(carpeta, archivo)
    resumen = resumen_tcx(ruta)
    if resumen:
        resumenes.append(resumen)

# Exportar archivos: .csv, .json, .sql

In [18]:
# Exportar a JSON
with open('resumen_entrenamientos.json', 'w', encoding='utf-8') as f:
    json.dump(resumenes, f, ensure_ascii=False, indent=4)

# Exportar a CSV
df_resumen = pd.DataFrame(resumenes)
df_resumen.to_csv('resumen_entrenamientos.csv', index=False, encoding='utf-8')

# Exportar a archivo .sql
with open('resumen_entrenamientos.sql', 'w', encoding='utf-8') as f:
    # Crear tabla
    f.write("""
IF OBJECT_ID('dbo.resumen_entrenamientos', 'U') IS NULL
BEGIN
    CREATE TABLE dbo.resumen_entrenamientos (
        Id INT IDENTITY(1,1) PRIMARY KEY,
        Archivo NVARCHAR(255),
        Tipo_deporte NVARCHAR(50),
        Inicio DATETIME,
        Fin DATETIME,
        Duracion_min FLOAT,
        Distancia_total_km FLOAT,
        Velocidad_promedio_kmh FLOAT,
        Calorias INT
    )
END
GO

""")

    # Insertar datos
    for r in resumenes:
        try:
            insert_line = (
                "INSERT INTO dbo.resumen_entrenamientos (Archivo, Tipo_deporte, Inicio, Fin, Duracion_min, Distancia_total_km, Velocidad_promedio_kmh, Calorias)\n"
                f"VALUES (N'{r['Archivo']}', N'{r['Tipo de deporte']}', '{r['Inicio']}', '{r['Fin']}', {r['Duración (min)']}, {r['Distancia total (km)']}, {r['Velocidad promedio (km/h)']}, {r['Calorías']});\n"
            )
            f.write(insert_line)
        except KeyError as e:
            print(f"❌ Faltan datos en una fila: {e}")
    f.write("GO\n")


## Se obtuvieron archivos de muestra, los archivos completos se utilizaran en analisis por medio de Power Bi y Proyectos de SQL